In [80]:
import pandas as pd
from scipy.stats import nbinom, poisson
import numpy as np
from joblib import Parallel, delayed

def truncNegBin_logCDF(y, n, p):
    f_zero = nbinom.pmf(0, n, p)
    if y > 0:
        return np.log((nbinom.cdf(y, n, p) - nbinom.cdf(0, n, p)) / (1 - f_zero))
    else:
        return np.log(0)
    
def log1mexp(x):
    if np.any((x < 0) & (~np.isnan(x))):
        raise ValueError("Inputs need to be non-negative!")
    return np.where(x <= np.log(2), np.log(-np.expm1(-x)), np.log1p(-np.exp(-x)))

In [71]:
#log.p	logical; if TRUE, probabilities p are given as log(p)
def qnbinom(p, nNbinom, pNbinom, log_p=False):
    # Convert p to array if it's a single value
    if not isinstance(p, (list, np.ndarray)):
        p = np.array([p])
    
    # Set log-probabilities (lower tail)
    n = len(p)
    if log_p:
        logp = p
    else:
        logp = np.log(p)
    
    # Set output and deal with special cases (outputs NA and Inf)
    quantiles = np.full(n, np.nan)
    nna = ~np.isnan(logp)
    nlogp = logp[nna]
    if len(nlogp) == 0:
        return quantiles
    
    quantiles[nna] = np.full(len(nna), np.inf)
    if np.min(nlogp) >= 0:
        return quantiles


    # NOCH ÄNDERN---------------------------------------------------------

    #mean = (n * (1-pNbinom))/pNbinom

    # Set log-CDF vector
    #lp_max = np.max(nlogp[nlogp < 0])
    #upper = int(mean + np.sqrt(mean * np.exp(-log1mexp(-lp_max)))) #Chebychev inequality
    upper = nbinom.ppf(0.999, nNbinom, pNbinom)
    logcdf = nbinom.logcdf(np.arange(upper+1), nNbinom, pNbinom)
    #---------------------------------------------------------------------

    # Compute output
    for i in range(n):
        if nna[i]:
            if logp[i] < 0:
                quantiles[i] = np.sum(logcdf < logp[i])
    
    # Return output
    if len(quantiles) == 1:
        return quantiles[0]
    else:
        return quantiles

#log.p	logical; if TRUE, probabilities p are given as log(p)    
def qnbinom_trunc(p, nNbinom, pNbinom, log_p=False):
    # Convert p to array if it's a single value
    if not isinstance(p, (list, np.ndarray)):
        p = np.array([p])
    
    # Set log-probabilities (lower tail)
    n = len(p)
    if log_p:
        logp = p
    else:
        logp = np.log(p)
    
    # Set output and deal with special cases (outputs NA and Inf)
    quantiles = np.full(n, np.nan)
    nna = ~np.isnan(logp)
    nlogp = logp[nna]
    if len(nlogp) == 0:
        return quantiles
    
    quantiles[nna] = np.full(len(nna), np.inf)
    if np.min(nlogp) >= 0:
        return quantiles


    # NOCH ÄNDERN---------------------------------------------------------

    #mean = (n * (1-pNbinom))/pNbinom

    # Set log-CDF vector
    #lp_max = np.max(nlogp[nlogp < 0])
    #upper = int(mean + np.sqrt(mean * np.exp(-log1mexp(-lp_max)))) #Chebychev inequality
    upper = nbinom.ppf(0.999, nNbinom, pNbinom)
    logcdf = nbinom.logcdf(np.arange(upper+1), nNbinom, pNbinom)
    logcdf = np.array([truncNegBin_logCDF(yi, nNbinom, pNbinom) for yi in range(1, int(upper)+1000)]) 
    #---------------------------------------------------------------------

    # Compute output
    for i in range(n):
        if nna[i]:
            if logp[i] < 0:
                quantiles[i] = np.sum(logcdf < logp[i]) + 1 #+1 because 0 is truncated
    
    # Return output
    if len(quantiles) == 1:
        return quantiles[0]
    else:
        return quantiles

In [77]:
def truncNegBin_CDF(y, n, p):
    f_zero = nbinom.pmf(0, n, p)
    if y > 0:
        return (nbinom.cdf(y, n, p) - nbinom.cdf(0, n, p)) / (1 - f_zero)
    else:
        return 0

def truncNegBin_PPF(x, n, p, epsilon=1e-6, max_iterations=100):
    # if f(0)=0 no truncation is needed
    if (1 - nbinom.pmf(0, n, p)) == 1:
        return nbinom.ppf(x, n, p)
    else:
        # Define the range of y where the solution might exist
        lower_bound = 0
        upper_bound = 1000000000  # Adjust this based on the expected range of y

        # Bisection method
        for _ in range(max_iterations):
            y = (lower_bound + upper_bound) / 2
            cdf_value = truncNegBin_CDF(y, n, p)

            if abs(cdf_value - x) < epsilon:
                return np.ceil(y)  # Found a good approximation

            if cdf_value < x:
                lower_bound = y
            else:
                upper_bound = y

        # Return the best approximation if max_iterations is reached
        return np.ceil(y)

def calculate_trunc_nbinom_quantile(quantile, n, p):
    return truncNegBin_PPF(quantile, n, p)

In [81]:
# calculation of quantiles
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies

# Example usage
mean = 1
var = 10

n = (mean**2) / (var - mean) # equivalent to r
p = mean / var

a = nbinom.ppf(quantiles, n, p)
b = qnbinom_trunc(quantiles, n, p)
trunc_nbinom_quantiles = Parallel(n_jobs=-1)(delayed(calculate_trunc_nbinom_quantile)(quantile, n, p) for quantile in quantiles) #fast way
c = np.array(trunc_nbinom_quantiles)

In [86]:
d = b == c

In [87]:
d[d == False]

array([], dtype=bool)